In [1]:
import os
import re
import pandas as pd
import numpy
import csv
import json

In [6]:

# Variable for the parent directory that contains all Java files
parent_directory = "./13decompiled/"  # Replace with the directory containing your Java files
csv_file = "/home/hfaheem/Documents/Datadrivennew/data-driven-analysis/methodoutput/apiToInvokeMap.csv"
output_dir = ""
# Define the class and method names you want to search for

In [7]:
column_names = [i for i in range(0, 100)]
df = pd.read_csv(csv_file, header=None, names=column_names)
# remove the first row
df = df.iloc[1:]
df

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
1,com.android.server.trust.TrustManagerService$1...,"com.android.server.trust.TrustManagerService, ...","com.android.server.LocalServices, getService, ...","android.os.Binder, getCallingUid, 0, I","com.android.server.trust.TrustManagerService, ...",com.android.server.trust.TrustManagerService$1...,"android.os.Binder, restoreCallingIdentity, 1, V","com.android.server.trust.TrustManagerService, ...","com.android.server.trust.TrustManagerService, ...","android.os.UserHandle, isCore, 1, Z",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,com.android.server.trust.TrustManagerService$1...,com.android.server.trust.TrustManagerService$1...,"com.android.server.trust.TrustManagerService, ...","com.android.server.trust.TrustManagerService, ...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,com.android.server.trust.TrustManagerService$1...,"com.android.server.trust.TrustManagerService, ...","com.android.server.trust.TrustManagerService, ...",com.android.server.trust.TrustManagerService$1...,"com.android.server.trust.TrustManagerService, ...","com.android.server.trust.TrustManagerService, ...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,com.android.server.trust.TrustManagerService$1...,"android.os.Bundle, putInt, 3, V","com.android.server.trust.TrustManagerService, ...",com.android.server.trust.TrustManagerService$1...,"com.android.server.trust.TrustManagerService, ...","android.os.Bundle, <init>, 1, V","com.android.server.trust.TrustManagerService, ...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,com.android.server.trust.TrustManagerService$1...,"com.android.server.trust.TrustManagerService, ...",com.android.server.trust.TrustManagerService$1...,"com.android.server.trust.TrustManagerService, ...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,com.android.server.trust.TrustManagerService$1...,"com.android.server.trust.TrustManagerService, ...",com.android.server.trust.TrustManagerService$1...,"com.android.server.trust.TrustManagerService, ...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,com.android.server.trust.TrustManagerService$1...,"com.android.server.trust.TrustManagerService, ...",com.android.server.trust.TrustManagerService$1...,com.android.server.trust.TrustManagerService$1...,"com.android.server.trust.TrustManagerService, ...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,com.android.server.trust.TrustManagerService$1...,"android.util.Log, w, 3, I","com.android.server.trust.TrustManagerService, ...",android.security.authorization.IKeystoreAuthor...,"com.android.server.trust.TrustManagerService, ...","android.content.Intent, putExtra, 3, Landroid/...","android.content.Intent, addFlags, 2, Landroid/...","android.security.Authorization, getService, 0,...","android.content.Intent, <init>, 2, V","com.android.server.trust.TrustManagerService, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,com.android.server.trust.TrustManagerService$1...,"com.android.server.trust.TrustManagerService, ...",com.android.server.trust.TrustManagerService$1...,"com.android.server.trust.TrustManagerService, ...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,com.android.server.trust.TrustManagerService$1...,"com.android.server.trust.TrustManagerService, ...",com.android.server.trust.TrustManagerService$1...,"com.android.server.trust.TrustManagerService, ...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:


def extract_method_code(file_path, method_name):
    method_code = []
    inside_method = False
    brace_count = 0
    method_signature_pattern = re.compile(rf'\b{method_name}\b\s*\([^)]*\)\s*\{{')  # Match method signature with curly brace

    with open(file_path, 'r') as file:
        for line in file:
            # Check if the line contains the method signature and starts the method definition
            if not inside_method and method_signature_pattern.search(line):

                inside_method = True
                method_code.append(line.strip())
                brace_count += line.count('{') - line.count('}')
                continue  # Proceed to the next line for further capture

            elif inside_method:
                method_code.append(line.strip())
                # Count opening and closing braces to track method boundaries
                brace_count += line.count('{') - line.count('}')

                # If brace count is zero, we've reached the end of the method
                if brace_count == 0:
                    break

    # Join the collected method lines and return as a string
    if method_code:
        return '\n'.join(method_code)

In [100]:
def extract_method_code_exact(file_path, method_name):
    method_code = []
    inside_method = False
    brace_count = 0

    # Loop through the file line by line
    with open(file_path, 'r') as file:
        for line in file:
            # Ensure we are matching exactly the method name along with 'public', 'protected', or 'private'
            if not inside_method and (('public' in line or 'protected' in line or 'private' in line) and f' {method_name}(' in line and '{' in line):
                inside_method = True
                method_code.append(line.strip())
                brace_count += line.count('{') - line.count('}')
                continue  # Proceed to the next line for further capture

            elif inside_method:
                method_code.append(line.strip())
                # Count opening and closing braces to track method boundaries
                brace_count += line.count('{') - line.count('}')

                # If brace count is zero, we've reached the end of the method
                if brace_count == 0:
                    break

    # Join the collected method lines and return as a string
    if method_code:
        return '\n'.join(method_code)
    else:
        return ""


# Function to find the specified classes and extract the methods from the decompiled files
def find_and_extract_methods(parent_dir, className, methodName, noParams, returnType):
    # Traverse the parent directory and search for specified classes and methods
    for root, dirs, files in os.walk(parent_dir):
        for file in files:
            if file.endswith(".java"):
                java_file_path = os.path.join(root, file)
                with open(java_file_path, 'r') as f:
                    file_content = f.read()

                if className in file_content:
                    # print(f"Class found: {className} in {java_file_path}")
                    # print(f"Extracting method: {methodName}")
                    method_code = extract_method_code_exact(java_file_path, methodName)
                    if method_code:
                        method_code+= "\n"
                        method_code += "-" * 80 + "\n"
                        # print(f"Class found: {className} in {java_file_path}")
                        # print(f"Method code for {methodName}:\n")
                        # print(method_code)
                        # print("-" * 80)
                        return method_code
                    else:
                        return ""


# Function to clean up synthetic method names
def clean_synthetic_method(method_name):
    # Match synthetic methods with -$$Nest$ prefix
    match = re.match(r"-\$\$Nest\$[mf](\w+)", method_name)
    if match:
        # Return the original method name without the prefix
        return match.group(1)
    return method_name

In [101]:
def print_method_details(cell, row_index):
    # Handle missing data or empty cells
    if pd.isna(cell) or not cell:
        print("Empty or invalid data.")
        return
    parts = cell.split(', ')
    if len(parts) < 4:  # Ensure there are enough parts to prevent errors
        print("Incomplete data.")
        return
    # print({
    #     'Class': parts[0],
    #     'Method': clean_synthetic_method(parts[1]),
    #     'Param Count': int(parts[2]),
    #     'Return Type': parts[3],
    #     'Row Index': row_index
    # })
    
    className = parts[0]
    methodName = clean_synthetic_method(parts[1])
    noParams = int(parts[2])
    returnType = parts[3]
    
    
    return find_and_extract_methods(parent_directory,className, methodName, noParams, returnType)
    
    



for index, row in df.head(1).iterrows():
    content = ""
    for cell in row:
        if not pd.isna(cell):
            content += (print_method_details(cell, index)) 
            # or ""
        else:
            break
    print(row[0])
    print(content)

com.android.server.trust.TrustManagerService$1, isDeviceLocked, 3, Z
public boolean isDeviceLocked(int i, int i2) throws RemoteException {
int callingUid = ITrustManager.Stub.getCallingUid();
if (isAppOrDisplayOnAnyVirtualDevice(callingUid, i2)) {
return false;
}
int handleIncomingUser = ActivityManager.handleIncomingUser(ITrustManager.Stub.getCallingPid(), callingUid, i, false, true, "isDeviceLocked", null);
long clearCallingIdentity = Binder.clearCallingIdentity();
try {
if (!TrustManagerService.this.mLockPatternUtils.isSeparateProfileChallengeEnabled(handleIncomingUser)) {
handleIncomingUser = TrustManagerService.this.resolveProfileParent(handleIncomingUser);
}
return TrustManagerService.this.isDeviceLockedInner(handleIncomingUser);
} finally {
Binder.restoreCallingIdentity(clearCallingIdentity);
}
}
--------------------------------------------------------------------------------
public final int resolveProfileParent(int i) {
long clearCallingIdentity = Binder.clearCallingIdentity()

In [82]:
df.iloc[index][0]

'com.android.server.trust.TrustManagerService$1, unregisterTrustListener, 2, V'

In [64]:
# Run the extraction for the specified parent directory and class-method map
find_and_extract_methods(parent_directory, "com.android.server.trust.TrustManagerService$1", 'isDeviceLocked',0,0)

Class found: com.android.server.trust.TrustManagerService$1 in ./13decompiled/sources/com/android/server/trust/TrustManagerService.java
Method code for isDeviceLocked:

public boolean isDeviceLocked(int i, int i2) throws RemoteException {
int callingUid = ITrustManager.Stub.getCallingUid();
if (isAppOrDisplayOnAnyVirtualDevice(callingUid, i2)) {
return false;
}
int handleIncomingUser = ActivityManager.handleIncomingUser(ITrustManager.Stub.getCallingPid(), callingUid, i, false, true, "isDeviceLocked", null);
long clearCallingIdentity = Binder.clearCallingIdentity();
try {
if (!TrustManagerService.this.mLockPatternUtils.isSeparateProfileChallengeEnabled(handleIncomingUser)) {
handleIncomingUser = TrustManagerService.this.resolveProfileParent(handleIncomingUser);
}
return TrustManagerService.this.isDeviceLockedInner(handleIncomingUser);
} finally {
Binder.restoreCallingIdentity(clearCallingIdentity);
}
}
--------------------------------------------------------------------------------


In [14]:
clean_synthetic_method("-$$Nest$mresolveProfileParent")

'resolveProfileParent'